In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# movies = pd.read_csv('data/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
# users = pd.read_csv('data/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
# ratings = pd.read_csv('data/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
training_set = pd.read_csv("../data/ml-100k/u1.base", delimiter="\t", header=None)
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [4]:
training_set = np.array(training_set, dtype="int")
testing_set = pd.read_csv("../data/ml-100k/u1.test", delimiter="\t", header=None)
testing_set = np.array(testing_set, dtype="int")

In [5]:
nb_users = int(max(max(training_set[:, 0]), max(testing_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(testing_set[:, 1])))

In [6]:
def convert(data):
    new_data = []
    for user_id in range(1, nb_users + 1):
        movie_ids = data[:, 1][data[:, 0] == user_id]
        rating_ids = data[:, 2][data[:, 0] == user_id]
        ratings = np.zeros(nb_movies)
        ratings[movie_ids - 1] = rating_ids
        new_data.append(list(ratings))

    return new_data

In [7]:
training_set = convert(training_set)
testing_set = convert(testing_set)

In [8]:
training_set = torch.FloatTensor(training_set)
testing_set = torch.FloatTensor(testing_set)

In [9]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [10]:
testing_set[testing_set == 0] = -1
testing_set[testing_set == 1] = 0
testing_set[testing_set == 2] = 0
testing_set[testing_set >= 3] = 1

In [11]:
class RBM(nn.Module):
    def __init__(self, nv, nh):
        super(RBM, self).__init__()
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)

    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)

    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)

    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [12]:
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

In [13]:
nb_epochs = 10
for epoch in range(1, nb_epochs + 1):
    train_loss = 0
    s = 0.0
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0, _ = rbm.sample_h(v0)
        for k in range(10):
            _, hk = rbm.sample_h(vk)
            _, vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk, _ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.0
    print(f"Epoch: {epoch}\t Loss: {train_loss/s}")

Epoch: 1	 Loss: 0.3466939628124237
Epoch: 2	 Loss: 0.24787235260009766
Epoch: 3	 Loss: 0.25384196639060974
Epoch: 4	 Loss: 0.2503244876861572
Epoch: 5	 Loss: 0.250525563955307
Epoch: 6	 Loss: 0.2503018081188202
Epoch: 7	 Loss: 0.24983848631381989
Epoch: 8	 Loss: 0.24632133543491364
Epoch: 9	 Loss: 0.2488655298948288
Epoch: 10	 Loss: 0.24947255849838257


In [14]:
test_loss = 0
s = 0.0
for id_user in range(nb_users):
    v = training_set[id_user : id_user + 1]
    vt = testing_set[id_user : id_user + 1]
    if len(vt[vt >= 0]) > 0:
        _, h = rbm.sample_h(v)
        _, v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
        s += 1.0
print(f"Test Loss: {test_loss/s}")

Test Loss: 0.23689661920070648
